In [23]:
import pandas as pd
import os
import re
import json

In [8]:
adjusted_forecast = pd.read_excel("C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal/ERCOT-Adjusted-Forecast.xlsb")

In [ ]:
def load_solution():
    with open("C:/Users/rundx/idaes-gtep/First decent run 4_23_long/dispatchable_investments.json", "r") as fil:
        dispatchable_dict = json.load(fil)
    print(dispatchable_dict)
    dispatchable_dict = {k:v for k,v in dispatchable_dict.items() if "investmentStage[3]" in k}
    print(dispatchable_dict)
load_solution()

{'investmentStage[1].genOperational.1': True, 'investmentStage[1].genOperational.2': True, 'investmentStage[1].genOperational.3': True, 'investmentStage[1].genOperational.4': True, 'investmentStage[1].genOperational.5': True, 'investmentStage[1].genOperational.6': True, 'investmentStage[1].genOperational.7': True, 'investmentStage[1].genOperational.8': True, 'investmentStage[1].genOperational.10': True, 'investmentStage[1].genOperational.11': True, 'investmentStage[1].genOperational.12': True, 'investmentStage[1].genOperational.13': True, 'investmentStage[1].genOperational.14': True, 'investmentStage[1].genOperational.15': True, 'investmentStage[1].genOperational.16': True, 'investmentStage[1].genOperational.18': True, 'investmentStage[1].genOperational.25': True, 'investmentStage[1].genOperational.26': True, 'investmentStage[1].genOperational.27': True, 'investmentStage[1].genOperational.28': True, 'investmentStage[1].genOperational.44': True, 'investmentStage[1].genOperational.45': T

In [ ]:
input_directory = "C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal"
output_directory = "C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal/Prescient"
def populate_generators(data_input_path, data_output_path):
    gen_df = pd.read_csv(input_directory + "/gen.csv")
    columns = gen_df.columns.to_list()
    columns.remove('Output_pct_2')
    columns.remove('Output_pct_3')
    columns.remove('Output_pct_4')
    columns.remove('HR_incr_2')
    columns.remove('HR_incr_3')
    columns.remove('HR_incr_4')
    gen_df["Fuel Price $/MMBTU"] = 1
    gen_df["Output_pct_0"]= 0
    gen_df["HR_avg_0"].fillna(0, inplace=True)
    gen_df["HR_incr_1"] = gen_df["fuel_cost3"]
    print(gen_df)
populate_generators(input_directory, output_directory)

0       7.378403
1      22.801280
2      22.801280
3      22.801280
4      22.801280
         ...    
576     0.000000
577     0.000000
578     0.000000
579     0.000000
580     0.000000
Name: HR_incr_1, Length: 581, dtype: float64


In [13]:
gen_df

NameError: name 'gen_df' is not defined